Pablo Jimeno - UPV/EHU 2016

In [ ]:
%run 'Utilities_main.ipynb'

redm_filedir = '%s/redMaPPer_DR12_v6.3.fits' % cat_dir
raw_redm = import_redMaPPer(redm_filedir)

In [ ]:
#-------------------------------------------------------------------
def import_Y_A_results(subsample_bins):
    """Import results"""    
    
    sample_metas = []
    Y_vals = []
    A_vals = []
    Y_errs = []
    A_errs = []
    
    for sub_bin in subsample_bins:
          
        rich_min, rich_max, z_min, z_max = sub_bin
        sample_tag = sub_tag(sub_bin)
        
        if (rich_min>=rich_max) or (z_min>=z_max):
            continue
    
        #-------------------------------------------------------------------
        ## Import meta & SZ data:       
        store_Y_A_file = '%s/SZ_values/Y_A_data_%s.pickle' % (data_dir, sample_tag)
        with open(store_Y_A_file, 'rb') as handle:
            store_Y_A_data = pickle.load(handle)
        
        #sub_meta : [rich_min, rich_max, z_min, z_max, N_clu, rich_mean, z_mean]
        sub_meta = store_Y_A_data[0]
        Y        = store_Y_A_data[1]
        A        = store_Y_A_data[2]
        Y_err    = store_Y_A_data[3]
        A_err    = store_Y_A_data[4]
        
        sample_metas.append(sub_meta)
        Y_vals.append(Y)
        A_vals.append(A)
        Y_errs.append(Y_err)
        A_errs.append(A_err)
                    
    sample_metas = np.array(sample_metas)
    Y_vals = np.array(Y_vals)
    A_vals = np.array(A_vals)
    Y_errs = np.array(Y_errs)
    A_errs = np.array(A_errs)
                
    return sample_metas, Y_vals, A_vals, Y_errs, A_errs
    

In [ ]:
#-------------------------------------------------------------------
def compute_mean_M(sub_meta, MR_slope, MR_scatter):

    rich_min, rich_max, z_min, z_max = sub_meta[0], sub_meta[1], sub_meta[2], sub_meta[3]

    rich_mask = (raw_redm['rich'] >= rich_min) & (raw_redm['rich'] < rich_max)
    z_mask = (raw_redm['photo_z'] >= z_min) & (raw_redm['photo_z'] < z_max)
    global_mask = rich_mask & z_mask

    R_sample = raw_redm['rich'][global_mask]
    
    M_subsample = 0.
    
    for Robs in R_sample:
        M_subsample += M_from_Robs(Robs, MR_slope, MR_scatter)
        
    mean_M_subsample = M_subsample/len(R_sample)
    
    return mean_M_subsample
    
    
#-------------------------------------------------------------------
def model_M_scale(sample_metas, Y_vals, A_vals, Y_errs, A_errs, nr_bins, nz_bins):
       
    MR_slope = 1.1
    MR_scatter = 0.2
    
    ## Plot results:
    markersize = 3.
    colors = ['k', 'b', 'r', 'g', 'c', 'y']           
        
    fig = plt.figure(figsize=(5, 4))   
    ax = fig.add_subplot(111)
    
    #ax.set_xlim([sample_metas[:,5].min() - 10, sample_metas[:,5].max() + 10])
    ax.set_xlabel(r'$\widetilde{M}$')
    
    ax.set_ylim([0.05, 150.])
    ax.set_ylabel(r'$\left<Y\right>$')
    ax.set_yscale("log")
    

    for z in range(nz_bins):

        color = colors[z]

        mean_Ms = np.array([compute_mean_M(meta, MR_slope, MR_scatter) for meta in sample_metas[z::nz_bins]])
        N = sample_metas[z::nz_bins, 4]
        Y = Y_vals[z::nz_bins]
        A = A_vals[z::nz_bins]
        Y_err = Y_errs[z::nz_bins]
        A_err = A_errs[z::nz_bins]
        #R_vector, Y_scale_vals, MR_slope_val, MR_slope_err = model_R_scale(Y, Y_err, R)
        label = 'z: [%.3f, %.3f)' % (sample_metas[z][2], sample_metas[z][3])#,MR_slope_val, MR_slope_err)

        ax.errorbar(mean_Ms, Y, yerr=Y_err, fmt='%s:'%color, marker='o', markersize=markersize, label=label)
        #ax.plot(R_vector, Y_scale_vals, color=color, alpha=0.3)
        
    ax.plot([],[], alpha=0.3, label=r'$\widetilde{M}\,^{5/3}$')
    ax.legend(loc=1, numpoints=1)

              
    ## Display figure:
    plot_file = '%s/SZ_modeling/M_scale.png' % plot_dir
    fig.savefig(plot_file, bbox_inches='tight')
    plt.close()
    
    
    
    
#-------------------------------------------------------------------

R_vals = [20, 30, 50, 100, 200]
#R_vals = [20, 300]
#R_vals = [20, 30, 45, 70, 100, 150, 300]

z_vals = [0.100, 0.250, 0.400, 0.550]
#z_vals = [0.080, 0.550]

subsample_bins, nr_bins, nz_bins = automatic_binning(R_vals, z_vals) 

## Run code:
def run_MR_relation_code():
    sample_metas, Y_vals, A_vals, Y_errs, A_errs = import_Y_A_results(subsample_bins)
    model_M_scale(sample_metas, Y_vals, A_vals, Y_errs, A_errs, nr_bins, nz_bins)
    
    
try:
    run_MR_relation_code()
except:
    %run 'Modelling_SZ.ipynb'
    run_MR_relation_code()

print '\nMR relation analysis completed.'